At first, we'll import necessary modules, read the data and represent first 5 rows of it:

In [4]:
import numpy as np
import pandas as pd
import matplotlib as mpl

In [5]:
train_csv_raw = pd.read_csv("/Users/daniiltekunov/Desktop/mhc/mhc_train.csv")

print(train_csv_raw[0:5])

        mhc   sequence      meas  pep_class
0  HLAA0101  AADFPGIAR  0.084687          0
1  HLAA0101  AADKAAAAY  0.638438          1
2  HLAA0101  AADSFATSY  0.599135          1
3  HLAA0101  AAFLDDNAF  0.084687          0
4  HLAA0101  AAGLPAIFV  0.084687          0


Now we'll pre-process this data to represent qualitative features space.

Firstly, we'll delete an ejection, that was detected after visual data analysis - element 90849 HLAE0101, that has an 'E' letter in its mhc code. Inasmuch as this element is just 0.0011% of the whole dataset, we'll remove it to reduce possible features space by coverting [0,0,0] feature to Boolean.

In [6]:
train_csv = train_csv_raw[train_csv_raw.mhc != "HLAE0101"]

After that we can start creating features space, where we'll use the following methods:

1) Data normalization

2) Changing the dimension of a vector space via One-hot encoding

We'll start with the protein structure:

* *HLA* - constant value

* *A* - depends on the structure of the protein, can be 'A' or 'B', after removing the ejection nominally Boolean

* *0101* - A four-digit number, that changes from 0101 to 9901